# CarNet


In [76]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')
%config InlineBackend.figure_format = 'retina'
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [3]:
import tensorflow as tf

# # Configure TensorFlow to use GPU memory growth before initializing any TensorFlow operations
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Set memory growth for each GPU
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("Memory growth enabled for GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before any TensorFlow operations
#         print(f"Failed to set memory growth: {e}")
# else:
#     print("No GPU devices found")

import tensorflow as tf
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)


PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


I0000 00:00:1730676868.581277      28 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730676868.629610      28 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730676868.631310      28 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


# Download Data

The dataset consists of 2 segments: 
* train: the metadata file contains the file names and the count of different objects in each image.
* score: the metadata file contains the file names of images to be used for the final predictions.

Target variables (in this order):
1. signal
2. vehicle

These target variables are defined as follows:
* signal =  traffic light + stop sign
* vehicle = car + bus + truck + train + motorcycle + bicycle + airplane + boat

In [4]:
!aws s3 cp s3://danylo-ucla/carnet_dataset.zip ./
!unzip -u -q carnet_dataset.zip

download: s3://danylo-ucla/carnet_dataset.zip to ./carnet_dataset.zip


In [104]:
train_metadata = pd.read_csv('carnet_dataset/train/metadata.csv')
train_metadata.head()

,car,bus,truck,train,motorcycle,bicycle,airplane,boat,traffic light,stop sign,vehicle,signal,file_name
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,000000000064.jpg
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,000000000073.jpg
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,000000000074.jpg
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,000000000081.jpg
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,000000000086.jpg


In [105]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_metadata = pd.read_csv('carnet_dataset/train/metadata.csv')
train_data, val_data = train_test_split(train_metadata, test_size=0.2, random_state=42)

# Create an instance of ImageDataGenerator with data augmentation for training
data_gen = ImageDataGenerator()

train_generator = data_gen.flow_from_dataframe(
    dataframe=train_data,
    directory='carnet_dataset/train/images',
    x_col='file_name',
    y_col=['signal', 'vehicle'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

val_generator = data_gen.flow_from_dataframe(
    dataframe=val_data,
    directory='carnet_dataset/train/images',
    x_col='file_name',
    y_col=['signal', 'vehicle'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)



Found 18582 validated image filenames.
Found 4646 validated image filenames.


In [107]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50, ResNet101, ConvNeXtBase
from tensorflow.keras.regularizers import l2
import numpy as np
from tensorflow.keras.metrics import MeanSquaredError

# Model creation functions
def create_resnet50_model():
    input_layer = Input(shape=(224, 224, 3), name='image_input')
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(2, activation='relu')(x)  # Output for vehicle and signal
    model = Model(inputs=input_layer, outputs=output)
    for layer in base_model.layers[-5:]:  # Unfreeze more layers
        layer.trainable = True
    return model

def create_resnet101_model():
    input_layer = Input(shape=(224, 224, 3), name='image_input')
    base_model = ResNet101(weights='imagenet', include_top=False, input_tensor=input_layer)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    output = Dense(2, activation='relu')(x)  # Output for vehicle and signal
    model = Model(inputs=input_layer, outputs=output)
    for layer in base_model.layers[-5:]:  # Unfreeze more layers
        layer.trainable = True
    return model

def create_resnet101_frozen_model():
    input_layer = Input(shape=(224, 224, 3), name='image_input')
    base_model = ResNet101(weights='imagenet', include_top=False, input_tensor=input_layer)
    base_model.trainable = False  # All layers frozen
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(2, activation='relu')(x)  # Output for vehicle and signal
    model = Model(inputs=input_layer, outputs=output)
    return model

def create_convnext_model():
    input_layer = Input(shape=(224, 224, 3), name='image_input')
    base_model = ConvNeXtBase(weights='imagenet', include_top=False, input_tensor=input_layer)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    output = Dense(2, activation='relu')(x)  # Output for vehicle and signal
    model = Model(inputs=input_layer, outputs=output)
    for layer in base_model.layers[-5:]:  # Unfreeze more layers
        layer.trainable = True
    return model

# Training function
def train_model(model, train_generator, val_generator, epochs=10):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='mean_squared_error',  # Using MSE as loss
        metrics=[MeanSquaredError(name='mse')]  # Using MSE as metric
    )
    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
        ]
    )
    return model

# Create and train ResNet50 model
resnet50_model = create_resnet50_model()
resnet50_model = train_model(resnet50_model, train_generator, val_generator, epochs=10)

# Create and train ResNet101 model
resnet101_model = create_resnet101_model()
resnet101_model = train_model(resnet101_model, train_generator, val_generator, epochs=10)

# Create and train ResNet101 model with all layers frozen
resnet101_frozen_model = create_resnet101_frozen_model()
resnet101_frozen_model = train_model(resnet101_frozen_model, train_generator, val_generator, epochs=10)

# Create and train ConvNeXt model
convnext_model = create_convnext_model()
convnext_model = train_model(convnext_model, train_generator, val_generator, epochs=10)


Epoch 1/10
581/581 ━━━━━━━━━━━━━━━━━━━━ 75s 110ms/step - loss: 8.5035 - mse: 8.5035 - val_loss: 5.5404 - val_mse: 5.5404 - learning_rate: 1.0000e-04
Epoch 2/10
581/581 ━━━━━━━━━━━━━━━━━━━━ 54s 93ms/step - loss: 5.9520 - mse: 5.9520 - val_loss: 5.2025 - val_mse: 5.2025 - learning_rate: 1.0000e-04
Epoch 3/10
581/581 ━━━━━━━━━━━━━━━━━━━━ 54s 93ms/step - loss: 5.4329 - mse: 5.4329 - val_loss: 4.9491 - val_mse: 4.9491 - learning_rate: 1.0000e-04
Epoch 4/10
581/581 ━━━━━━━━━━━━━━━━━━━━ 54s 93ms/step - loss: 4.8704 - mse: 4.8704 - val_loss: 5.2901 - val_mse: 5.2901 - learning_rate: 1.0000e-04
Epoch 5/10
581/581 ━━━━━━━━━━━━━━━━━━━━ 54s 93ms/step - loss: 4.7669 - mse: 4.7669 - val_loss: 4.8501 - val_mse: 4.8501 - learning_rate: 1.0000e-04
Epoch 6/10
581/581 ━━━━━━━━━━━━━━━━━━━━ 54s 93ms/step - loss: 4.5581 - mse: 4.5581 - val_loss: 4.9092 - val_mse: 4.9092 - learning_rate: 1.0000e-04
Epoch 7/10
581/581 ━━━━━━━━━━━━━━━━━━━━ 54s 93ms/step - loss: 4.1296 - mse: 4.1296 - val_loss: 4.8650 - val_mse

In [110]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Define a function to calculate weighted ensemble predictions
def ensemble_predict(models, data, weights):
    predictions = [model.predict(data) for model in models]
    predictions = np.array(predictions)
    # Compute weighted average of the predictions
    avg_predictions = np.average(predictions, axis=0, weights=weights)
    return avg_predictions

val_mse_resnet50 = 4.845  
val_mse_resnet101 = 5.16  
val_mse_resnet101_frozen = 4.6442  
val_mse_convnext = 4.2
# Convert MSE to weights (lower MSE means higher weight)
total_mse = val_mse_resnet50 + val_mse_resnet101 + val_mse_resnet101_frozen + val_mse_convnext
weights = [
    1 - (val_mse_resnet50 / total_mse),
    1 - (val_mse_resnet101 / total_mse),
    1 - (val_mse_resnet101_frozen / total_mse),
    1 - (val_mse_convnext / total_mse)
]

# Normalize weights to sum to 1
weights /= np.sum(weights)

# Make ensemble predictions using the refined strategy
ensemble_preds = []

validation_steps = val_generator.n // val_generator.batch_size
for i in range(validation_steps):
    batch_data, _ = val_generator[i]
    predictions = ensemble_predict(
        [resnet50_model, resnet101_model, resnet101_frozen_model, convnext_model], 
        batch_data, 
        weights
    )
    ensemble_preds.append(predictions)

# Concatenate all batch predictions
ensemble_preds = np.vstack(ensemble_preds)

# Evaluate the ensemble model
val_labels = []

for i in range(validation_steps):
    _, batch_labels = val_generator[i]
    val_labels.append(batch_labels)

val_labels = np.vstack(val_labels)

# Calculate the ensemble's MSE manually
mse_signal = mean_squared_error(val_labels[:, 0], ensemble_preds[:, 0])
mse_vehicle = mean_squared_error(val_labels[:, 1], ensemble_preds[:, 1])
ensemble_mse = 0.5 * (mse_signal + mse_vehicle)

print(f"Weighted Ensemble MSE: {ensemble_mse}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler

# Standardize the additional columns
numeric_columns = ['car', 'bus', 'truck', 'train', 'motorcycle', 'bicycle', 'airplane', 'boat', 'traffic light', 'stop sign']
scaler = StandardScaler()
train_metadata[numeric_columns] = train_metadata[numeric_columns]

# # Define Image Data Generator
# data_gen = ImageDataGenerator(
#     rescale=1.0/255.0,
#     validation_split=0.2,
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )
# Define Image Data Generator for training images with basic preprocessing
data_gen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.25) 

# Function to create data generator
def create_image_gen(df, subset):
    return data_gen.flow_from_dataframe(
        dataframe=df,
        directory='carnet_dataset/train/images',
        x_col='file_name',
        y_col=numeric_columns,
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset=subset,
        shuffle=True
    )

# Create train and validation generators
train_image_gen = create_image_gen(train_metadata, 'training')
val_image_gen = create_image_gen(train_metadata, 'validation')

# Define model
def create_model():
    input_layer = Input(shape=(224, 224, 3), name='image_input')
    base_model = ResNet101(weights='imagenet', include_top=False, input_tensor=input_layer)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='gelu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    output = Dense(10, activation='relu')(x)  # Output for 10 individual classes

    model = Model(inputs=input_layer, outputs=output)

   
    for layer in base_model.layers[-5:]:  # Unfreeze more layers
        layer.trainable = True
    return model

model = create_model()

# Custom loss function to calculate MSE of derived signals and vehicles
def custom_loss(y_true, y_pred):
    # Ensure both y_true and y_pred are in float32 format
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Derived predictions
    pred_signal = y_pred[:, 8] + y_pred[:, 9]  # traffic light + stop sign
    pred_vehicle = tf.reduce_sum(y_pred[:, :8], axis=1)  # sum of car, bus, truck, etc.

    # Derived actual values
    true_signal = y_true[:, 8] + y_true[:, 9]
    true_vehicle = tf.reduce_sum(y_true[:, :8], axis=1)

    # Calculate MSE for signals and vehicles
    mse_signal = tf.reduce_mean(tf.square(true_signal - pred_signal))
    mse_vehicle = tf.reduce_mean(tf.square(true_vehicle - pred_vehicle))

    # Return average MSE as per given formula
    return 0.5 * mse_signal + 0.5 * mse_vehicle

# Compile the model with the custom loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=custom_loss,
              metrics=[custom_loss])

# Train the model
history = model.fit(
    train_image_gen,
    validation_data=val_image_gen,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
    ]
)


Found 17421 validated image filenames.
Found 5807 validated image filenames.
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/50


/app/conda/envs/collegium/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


545/545 ━━━━━━━━━━━━━━━━━━━━ 124s 196ms/step - custom_loss: 9.0125 - loss: 9.0112 - val_custom_loss: 7.7105 - val_loss: 7.7057 - learning_rate: 1.0000e-04
Epoch 2/50
545/545 ━━━━━━━━━━━━━━━━━━━━ 94s 172ms/step - custom_loss: 8.0842 - loss: 8.0785 - val_custom_loss: 8.7804 - val_loss: 8.7623 - learning_rate: 1.0000e-04
Epoch 3/50
545/545 ━━━━━━━━━━━━━━━━━━━━ 94s 173ms/step - custom_loss: 8.1403 - loss: 8.1404 - val_custom_loss: 8.0969 - val_loss: 8.0998 - learning_rate: 1.0000e-04
Epoch 4/50
545/545 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - custom_loss: 7.8556 - loss: 7.8576
Epoch 4: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
545/545 ━━━━━━━━━━━━━━━━━━━━ 94s 173ms/step - custom_loss: 7.8554 - loss: 7.8574 - val_custom_loss: 8.2168 - val_loss: 8.2017 - learning_rate: 1.0000e-04
Epoch 5/50
545/545 ━━━━━━━━━━━━━━━━━━━━ 94s 173ms/step - custom_loss: 7.7519 - loss: 7.7538 - val_custom_loss: 7.4125 - val_loss: 7.4050 - learning_rate: 1.0000e-05
Epoch 6/50
545/545 ━━━━━━━━━━

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
import numpy as np

# Define custom loss function with explicit casting
def average_mse(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # Split predictions and targets into signal and vehicle counts
    y_true_signal, y_true_vehicle = y_true[:, 0], y_true[:, 1]
    y_pred_signal, y_pred_vehicle = y_pred[:, 0], y_pred[:, 1]
    
    # Calculate MSE for each target manually
    mse_signal = tf.reduce_mean(tf.square(y_true_signal - y_pred_signal))
    mse_vehicle = tf.reduce_mean(tf.square(y_true_vehicle - y_pred_vehicle))
    
    # Return the average of both MSEs
    return 0.5 * mse_signal + 0.5 * mse_vehicle

# Model Creation Function
def create_model():
    input_layer = Input(shape=(224, 224, 3))
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=input_layer)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='gelu')(x)
    x = Dropout(0.4)(x)  # Dropout layer for regularization
    output = Dense(2, activation='linear')(x)

    model = Model(inputs=input_layer, outputs=output)
    
    # Freeze the layers initially and gradually unfreeze more layers in training
    for layer in base_model.layers:
        layer.trainable = False
    for layer in base_model.layers[-10:]:  # Unfreeze more layers
        layer.trainable = True

    # Custom learning rate schedule
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.0001,
        decay_steps=10000,
        decay_rate=0.96,
        staircase=True
    )

    model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=lr_schedule),
                  loss=average_mse,
                  metrics=[average_mse])
    return model

# Create a single model
model = create_model()

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=6,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
    ]
)




Training model 1/3


/app/conda/envs/collegium/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2


I0000 00:00:1730501819.177231     200 service.cc:146] XLA service 0x7f4b20053ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730501819.177256     200 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5


  2/581 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - average_mse: 10.8712 - loss: 10.8712 

I0000 00:00:1730501832.561072     200 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


581/581 ━━━━━━━━━━━━━━━━━━━━ 159s 231ms/step - average_mse: 9.3788 - loss: 9.3797 - val_average_mse: 10.5667 - val_loss: 10.4482 - learning_rate: 1.0000e-04
Epoch 2/2
581/581 ━━━━━━━━━━━━━━━━━━━━ 46s 79ms/step - average_mse: 8.9817 - loss: 8.9835 - val_average_mse: 10.5904 - val_loss: 10.2493 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 2.
Training model 2/3
Epoch 1/2
581/581 ━━━━━━━━━━━━━━━━━━━━ 81s 107ms/step - average_mse: 9.4124 - loss: 9.4027 - val_average_mse: 9.4244 - val_loss: 9.3614 - learning_rate: 1.0000e-04
Epoch 2/2
581/581 ━━━━━━━━━━━━━━━━━━━━ 46s 79ms/step - average_mse: 8.6443 - loss: 8.6460 - val_average_mse: 8.3624 - val_loss: 8.3713 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 2.
Training model 3/3
Epoch 1/2
581/581 ━━━━━━━━━━━━━━━━━━━━ 82s 107ms/step - average_mse: 9.5500 - loss: 9.5508 - val_average_mse: 8.9538 - val_loss: 8.9950 - learning_rate: 1.0000e-04
Epoch 2/2
581/581 ━━━━━━━━━━━━━

In [120]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Predict without using batches
# Load all images from score data
# Load all images from score data
score_data = score_data_gen.flow_from_dataframe(
    dataframe=score_metadata,
    directory='carnet_dataset/score/images',
    x_col='file_name',
    target_size=(224, 224),
    batch_size=score_metadata.shape[0],  # Load all images in one batch
    class_mode=None,
    shuffle=False
)

# Load all data in one go using __getitem__ to get the first (and only) batch
batch_data = score_data[0]

# Make predictions
ensemble_preds = ensemble_predict(
    [resnet50_model, resnet101_model, resnet101_frozen_model, convnext_model], 
    batch_data, 
    weights
)

# Create a DataFrame for predictions
score_y_hat = pd.DataFrame(ensemble_preds, columns=['signal', 'vehicle'])



Found 5766 validated image filenames.
181/181 ━━━━━━━━━━━━━━━━━━━━ 16s 87ms/step
181/181 ━━━━━━━━━━━━━━━━━━━━ 32s 175ms/step
181/181 ━━━━━━━━━━━━━━━━━━━━ 28s 156ms/step
181/181 ━━━━━━━━━━━━━━━━━━━━ 57s 318ms/step


# Saving the model and predictions

In [121]:
# Save your predictions on the Score segment as a Pandas data frame into a variable named `score_y_hat`.
# The data frame should contain 2 columns: signal and vehicle.
# The following is an EXAMPLE of what this data frame could be.

# Assert checks for the final predictions
assert type(score_y_hat) == pd.DataFrame
assert score_y_hat.shape == (score_metadata.shape[0], 2)
assert (score_y_hat.columns == ['signal', 'vehicle']).all()

# Save the predictions as a parquet file
import os 
model_dir = 'carnet_model'
os.makedirs(model_dir, exist_ok=True)
score_y_hat.to_parquet(f'{model_dir}/score_y_hat.parquet')


In [123]:
# Save each model's definition (JSON) and weights (H5)
for model, name in zip(
    [resnet50_model, resnet101_model, resnet101_frozen_model, convnext_model], 
    ['resnet50', 'resnet101', 'resnet101_frozen', 'convnext']
):
    # Save model architecture
    model_json = model.to_json()
    with open(f'{model_dir}/{name}_model.json', 'w') as json_file:
        json_file.write(model_json)
    
    # Save model weights with the required `.weights.h5` extension
    model.save_weights(f'{model_dir}/{name}_weights.weights.h5')


In [27]:
# Next, save each model's defition (JSON file) and parameters (H5 file).

In [28]:
# You now have the following files to be submit:
# 1. This notebook and any other Python code you used to train the final model.
# 2. definitions of all trained models
# 3. parameters of all trained models
# 4. score_y_hat.parquet - the model's output on the score segment